In [6]:
#!/usr/bin/env python

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

# Set data (4.2)

X = np.loadtxt('question-x.txt').astype(np.float32)
Y = np.loadtxt('question-y.txt')
N = Y.size
Y2 = np.zeros(8 * N).reshape(N,8).astype(np.float32)
for i in range(N):
    Y2[i,Y[i]] = 1.0

index = np.arange(N)
xtrain = X[index[index % 2 == 0],:]
ytrain = Y2[index[index % 2 == 0],:] #教師信号、3次元ベクトル
xtest = X[index[index % 2 != 0],:]
yans = Y[index[index % 2 != 0]]
#(4.2)ここまで

# Define model(4.3)

class IrisChain(Chain): #Chainのクラスを設定する
    def __init__(self):
        super(IrisChain, self).__init__(
            l1=L.Linear(66,100), #Linearクラスで線形変換を行う
            l2=L.Linear(100,8),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)
        # その値を二乗平均誤差の関数 F.mean_squared_error に渡して
        # 損失を求める

    def fwd(self,x):
         h1 = F.sigmoid(self.l1(x))
         # 出力を活性化関数 F.sigmoid に与える
         h2 = self.l2(h1)
         return h2

# Initialize model
#パラメータの学習 10000回パラメータを更新してパラメータを求める
model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

# Learn

for i in range(10000):
    x = Variable(xtrain)
    y = Variable(ytrain)
    model.zerograds()
    loss = model(x,y)
    loss.backward()
    optimizer.update()

# Test
# テストデータで評価
xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)

ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    print((ans[i,:], cls))
    if cls == yans[i]:
        ok += 1
        print(i * 2 + 3, "=", cls)

print(ok, "/", nrow, " = ", (ok * 1.0)/nrow)
#nrow:テストデータの総数,ok:正解数なので、
#正解率が出力される
#(4.3)


/Users/rui/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(array([ 0.31139854,  0.30431119,  0.013026  , -0.05426969,  0.17834954,
        0.02482321,  0.27284193,  0.01037691], dtype=float32), 0)
3 = 0
(array([ 0.15420678,  0.18874559,  0.57184541,  0.06727538, -0.00153835,
        0.16937324, -0.06527616,  0.01480098], dtype=float32), 2)
5 = 2
(array([ 0.18511745, -0.09198518,  0.12398667,  0.12921618,  0.80250633,
        0.0183965 , -0.05619057, -0.01181512], dtype=float32), 4)
7 = 4
(array([ 0.0754445 ,  0.01899652,  0.04119507,  0.52938235,  0.10369094,
        0.00603598,  0.22248022,  0.01637433], dtype=float32), 3)
9 = 3
(array([ 0.44496191,  0.01622893,  0.0981632 ,  0.04306305,  0.22939228,
        0.08471224,  0.08301716,  0.0066234 ], dtype=float32), 0)
11 = 0
(array([ 0.10264274, -0.03748144, -0.03270993,  0.10175081,  0.07654984,
        0.19906425,  0.54399157,  0.01214908], dtype=float32), 6)
(array([ 0.12370754,  0.13499625,  0.05583488,  0.37627271,  0.17896773,
        0.00780233,  0.22539179, -0.00711249], dtype=float32),